In [3]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
__author__ = "yan bin"
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.pipeline import Pipeline,PipelineModel
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import *
from pyspark.sql.functions import *  
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit,CrossValidator
from pandas import *
import matplotlib as mpl
import time
from numpy import arange,logspace
import sys
mpl.use("Agg")
class Evaluator(object):
    __spark = 0
    __sc = 0
    def __init__(self,recomPath,master="local", appName="evaluator"):
        conf = (SparkConf().setAppName(appName)
                .setMaster(master))
        self.__spark = (SparkSession.builder
                .config(conf=conf)
                .enableHiveSupport()
                .getOrCreate())
        self.__sc = self.__spark.sparkContext
        self.recomPath = recomPath
    
    def getContext(self):
        return (self.__spark, self.__sc)
        
    def getRecommendation(self,userId):
        spark.sc = self.getContext()
        recomDf = spark.read.csv(self.recomPath)
        recomUser = recomDf.filter(recomDf.userId==userId).select('itemID')
        return recomUser
    
#     def getTrue(self):
        
    def recall(train,test,N):
        hit = 0
        all = 0 
        for user in train.keys():
            tu = test[user] if test.get(user) else {}
            rank = getRecommendation(user,N)
            for item,pui in rank:
                if item in tu:
                    hit +=1
            all +=len(tu)
        return hit/(all * 1.0)

    def precision (train,test,N):
        hit = 0
        all = 0 
        for user in train.keys():
            tu = test[user] if test.get(user) else {}
            rank = getRecommendation(user,N)
            for item,pui in rank:
                if tu and item in tu:
                    hit +=1
            all += N
        return hit*1.0/(all*1.0)

    def coverage(train,test,N):
        recommend_items =set()
        all_items = set()
        for user in train.keys():
            for item in train[user].keys():
                all_items.add(item)
            rank = getRecommendation(user,N)
            for item,pui in rank:
                recommend_items.add(item)
        return len(recommend_items)/(len(all_items)*1.0)

    def popularity(train, test, N):
        item_popularity = dict()
        for user, items in train.items():
            for item in items:
                if item not in item_popularity:
                    item_popularity[item] = 0
                item_popularity[item] +=1

        ret = 0
        n = 0
        for user,items in train.items():
            rank = getRecommendation(user,N)
            for item,pui in rank:
                ret += math.log(1+item_popularity[item])
                n+=1
        ret /= n*1.0
        return ret